## Stage 1: Install all dependencies and setting up the environment

In [1]:
#!apt-get install python-dev python-snappy

In [3]:
!pip install tensorflow-data-validation

     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 19.1 MB 1.3 MB/s 
     |████████████████████████████████| 9.8 MB 37.7 MB/s 
     |████████████████████████████████| 294 kB 50.9 MB/s 
     |████████████████████████████████| 62 kB 722 kB/s 
     |████████████████████████████████| 249 kB 46.1 MB/s 
     |████████████████████████████████| 829 kB 37.4 MB/s 
     |████████████████████████████████| 2.3 MB 19.8 MB/s 
     |████████████████████████████████| 151 kB 49.7 MB/s 
     |████████████████████████████████| 144 kB 54.4 MB/s 
     |████████████████████████████████| 255 kB 56.3 MB/s 
     |████████████████████████████████| 169 kB 51.8 MB/s 
     |████████████████████████████████| 267 kB 46.1 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 183 kB 45.9 MB/s 
     |████████████████████████████████| 173 kB 38.9 MB/s 
     |████████████████████████████████| 435 kB 49.6 MB/s 
     |█████████████

## Stage 2: Import project dependencies

In [3]:
#!pip install google-api-core

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

## Stage 3: Simple dataset analysis

In [2]:
dataset = pd.read_csv("pollution_small.csv")

In [3]:
dataset.shape

(2188, 5)

In [4]:
training_data = dataset[:1600]

In [5]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [6]:
test_set = dataset[1600:]

In [7]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Stage 3: Data analysis and validation with TFDV

### Generate training data statistics

In [8]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

### Infering the schema

In [9]:
schema = tfdv.infer_schema(statistics=train_stats)

In [10]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calculate test set statistics

In [11]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

## Stage 4: Compare test statistics with the Schema

### Checking for anomalies in new data

In [12]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Displaying all detected anomalies

- Integer larger than 10
- STRING type when expected INT type
- FLOAT type when expected INT type
- Integer smaller than 0

In [13]:
tfdv.display_anomalies(anomalies)

### New data WITH anomalies

In [14]:
test_set_copy = test_set.copy()

In [15]:
test_set_copy.drop("soot", axis=1, inplace=True)

### Statistics based on data with anomalies

In [16]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [17]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [18]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Stage 5: Prepare the schema for Serving

In [19]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Removing a target column from the Serving schema

In [20]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Checking for anomalies between the SERVING environment and new test set

In [21]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [22]:
tfdv.display_anomalies(serving_env_anomalies)

## Freezing the schema:

In [23]:
tfdv.write_schema_text(schema=schema,output_path="pollution_schema.pbtxt")

In [24]:
tfdv.visualize_statistics(train_stats)